In [64]:
# !pip install dlib
# !pip install face_recognition
# !pip install keras
# !pip install tensorflow
# !pip install opencv-python

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import face_recognition
import matplotlib.image as mpimg
%matplotlib notebook

from PIL import Image

In [2]:
from keras.models import load_model
import numpy as np
import cv2

In [61]:
emotion_dict= {'Angry': 0, 'Sad': 5, 'Neutral': 4, 'Disgust': 1, 'Surprise': 6, 'Fear': 2, 'Happy': 3}
label_map = dict((v,k) for k,v in emotion_dict.items()) 

model = load_model("emotion_detector_models/model_v6_23.hdf5")

In [53]:
filename = "../images/_1578234493687243_r.jpg" # Face: 3/3; Emotion: 0/3
# filename = "../images/_1576452583705107_r.jpg" # Face: 1/1; Emotion: 0/1
# filename = "../images/_1585675653429775_r.jpg" # Face: 3/3; Emotion: 1/3

In [54]:
# Read Images 
img = mpimg.imread(filename) 
  
# Output Images 
pil_image = Image.fromarray(img)
pil_image.show()

In [55]:
image = face_recognition.load_image_file(filename)
#face_landmarks_list = face_recognition.face_landmarks(image)

In [56]:
# Face Location
face_locations = face_recognition.face_locations(image)
face_locations
# [(top, left, bottom, right)]

[(268, 296, 397, 167), (167, 494, 322, 339), (355, 450, 462, 342)]

In [57]:
face = []
for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))
    
    height = abs(top-bottom)
    width = abs(left-right)
    
#     top -= int(0.10*height) # Top extra by 10%
#     bottom += int(0.10*height) # Bottom extra by 10%
#     left -= int(0.05*width) # Left extra by 5%
#     right += int(0.05*width) # Right extra by 5%

#     top -= 150
#     bottom += 100
#     left -= 50
#     right += 50
    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    face.append(face_image)
    
#     plt.imshow(face_image)
    pil_image = Image.fromarray(face_image)
    pil_image.show()

A face is located at pixel location Top: 268, Left: 167, Bottom: 397, Right: 296
A face is located at pixel location Top: 167, Left: 339, Bottom: 322, Right: 494
A face is located at pixel location Top: 355, Left: 342, Bottom: 462, Right: 450


# Emotion Prediction

In [58]:
resizedFace = []
for face_image in face:
    print(face_image.shape)
    
    # resizing the image to (1, 48, 48, 1)
    face_image = cv2.resize(face_image, (48,48))
    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    face_image = np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])
    resizedFace.append(face_image)

(129, 129, 3)
(155, 155, 3)
(107, 108, 3)


In [59]:
for face_image in resizedFace:
    predicted_class = np.argmax(model.predict(face_image))
    predicted_label = label_map[predicted_class]
    print(predicted_label)

Surprise
Surprise
Fear


# Process All Images

In [3]:
import os

directory = "../images/"
files = []
for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
        files.append(os.path.join(directory, filename))
print(len(files))

1303


In [73]:
import pandas as pd
df = pd.DataFrame(columns = ["File","Count","Location","Emotions"])
df["File"] = files
df.head()

,File,Count,Location,Emotions
0,../images/0.jpg,NaN,NaN,NaN
1,../images/33474154_1569286032341471_r.jpg,NaN,NaN,NaN
2,../images/36835844_1576190421716323_r.jpg,NaN,NaN,NaN
3,../images/38668776_1583694474494563_r.jpg,NaN,NaN,NaN
4,../images/39160502_1571973681329786_r.jpg,NaN,NaN,NaN


In [77]:
for index, row in df.iterrows():
    filePath = row["File"]
    
    # Read Image
    print("\n"+str(index)+": "+filePath+"\n")
    img = mpimg.imread(filePath)
    
#     # Output Images 
#     pil_image = Image.fromarray(img)
#     pil_image.show()
    
    # Load Image
    image = face_recognition.load_image_file(filePath)

    # Face Location
    face_locations = face_recognition.face_locations(image)
    face_locations
    # [(top, left, bottom, right)]
    
    row["Count"] = len(face_locations)
    row["Location"] = face_locations
    
    face = []
    for face_location in face_locations:
        # Print the location of each face in this image
        top, right, bottom, left = face_location
        print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

        height = abs(top-bottom)
        width = abs(left-right)

#         top -= int(0.10*height) # Top extra by 10%
#         bottom += int(0.10*height) # Bottom extra by 10%
#         left -= int(0.05*width) # Left extra by 5%
#         right += int(0.05*width) # Right extra by 5%

        # You can access the actual face itself like this:
        face_image = image[top:bottom, left:right]
#         pil_image = Image.fromarray(face_image)
#         pil_image.show()
#         face.append(face_image)
#         print(face_image.shape)

        # Resizing the image to (1, 48, 48, 1)
        face_image = cv2.resize(face_image, (48,48))
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
        face_image = np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])
#         resizedFace.append(face_image)

        # Emotion Prediction
        predicted_class = np.argmax(model.predict(face_image))
        predicted_label = label_map[predicted_class]
        
        face.append(predicted_label)
        
        row["Emotions"] = face


0: ../images/0.jpg


1: ../images/33474154_1569286032341471_r.jpg

A face is located at pixel location Top: 142, Left: 439, Bottom: 365, Right: 662
A face is located at pixel location Top: 259, Left: 91, Bottom: 367, Right: 199

2: ../images/36835844_1576190421716323_r.jpg


3: ../images/38668776_1583694474494563_r.jpg


4: ../images/39160502_1571973681329786_r.jpg

A face is located at pixel location Top: 1047, Left: 869, Bottom: 1845, Right: 1668
A face is located at pixel location Top: 1623, Left: 2709, Bottom: 1697, Right: 2783
A face is located at pixel location Top: 1880, Left: 2817, Bottom: 1954, Right: 2891
A face is located at pixel location Top: 1832, Left: 489, Bottom: 2017, Right: 675
A face is located at pixel location Top: 558, Left: 2846, Bottom: 825, Right: 3113
A face is located at pixel location Top: 1569, Left: 2833, Bottom: 1659, Right: 2922
A face is located at pixel location Top: 587, Left: 379, Bottom: 855, Right: 646
A face is located at pixel location Top: 165


42: ../images/42171146_1569256704822992_r.jpg


43: ../images/42180138_1572020219327876_r.jpg


44: ../images/42190662_1595705100155792_r.jpg


45: ../images/42200342_1569130698225423_r.jpg


46: ../images/42209786_1569405548140206_r.jpg

A face is located at pixel location Top: 288, Left: 356, Bottom: 442, Right: 511
A face is located at pixel location Top: 93, Left: 241, Bottom: 316, Right: 464

47: ../images/42224944_1569244353780450_r.jpg


48: ../images/42259690_1569392054248801_r.jpg

A face is located at pixel location Top: 76, Left: 76, Bottom: 461, Right: 461

49: ../images/42260568_1569356335111991_r.jpg

A face is located at pixel location Top: 605, Left: 1529, Bottom: 1067, Right: 1991

50: ../images/42262694_1569360930851723_r.jpg

A face is located at pixel location Top: 357, Left: 0, Bottom: 911, Right: 479

51: ../images/42263526_1569363390209946_r.jpg

A face is located at pixel location Top: 170, Left: 262, Bottom: 222, Right: 314

52: ../images/42302774_156950497375


91: ../images/42579064_1570557749460808_r.jpg

A face is located at pixel location Top: 0, Left: 1149, Bottom: 937, Right: 2108

92: ../images/42597790_1570630517933781_r.jpg

A face is located at pixel location Top: 1224, Left: 2111, Bottom: 2023, Right: 2910

93: ../images/42607856_1570655453109751_r.jpg

A face is located at pixel location Top: 192, Left: 414, Bottom: 415, Right: 637

94: ../images/42634316_1570749324411821_r.jpg

A face is located at pixel location Top: 82, Left: 52, Bottom: 350, Right: 319

95: ../images/42636806_1570763038705898_r.jpg

A face is located at pixel location Top: 113, Left: 345, Bottom: 188, Right: 419
A face is located at pixel location Top: 113, Left: 411, Bottom: 188, Right: 486

96: ../images/42643768_1570925355316184_r.jpg

A face is located at pixel location Top: 211, Left: 210, Bottom: 340, Right: 339
A face is located at pixel location Top: 184, Left: 442, Bottom: 339, Right: 597
A face is located at pixel location Top: 67, Left: 296, Bottom

A face is located at pixel location Top: 30, Left: 63, Bottom: 105, Right: 137

133: ../images/43067012_1572485814955266_r.jpg

A face is located at pixel location Top: 156, Left: 315, Bottom: 218, Right: 377
A face is located at pixel location Top: 94, Left: 391, Bottom: 156, Right: 453
A face is located at pixel location Top: 107, Left: 20, Bottom: 159, Right: 72
A face is located at pixel location Top: 135, Left: 183, Bottom: 198, Right: 246

134: ../images/43070222_1572876658177328_r.jpg

A face is located at pixel location Top: 281, Left: 218, Bottom: 343, Right: 280
A face is located at pixel location Top: 218, Left: 453, Bottom: 262, Right: 496
A face is located at pixel location Top: 96, Left: 254, Bottom: 171, Right: 328
A face is located at pixel location Top: 132, Left: 400, Bottom: 175, Right: 443
A face is located at pixel location Top: 234, Left: 515, Bottom: 285, Right: 567

135: ../images/43077642_1572548591567158_r.jpg

A face is located at pixel location Top: 889, Lef

A face is located at pixel location Top: 1529, Left: 962, Bottom: 1619, Right: 1051
A face is located at pixel location Top: 1211, Left: 225, Bottom: 1301, Right: 315
A face is located at pixel location Top: 1573, Left: 1752, Bottom: 1680, Right: 1859
A face is located at pixel location Top: 1441, Left: 2170, Bottom: 1549, Right: 2277
A face is located at pixel location Top: 1609, Left: 1350, Bottom: 1699, Right: 1439
A face is located at pixel location Top: 1211, Left: 703, Bottom: 1301, Right: 792
A face is located at pixel location Top: 1394, Left: 2002, Bottom: 1501, Right: 2110
A face is located at pixel location Top: 1231, Left: 424, Bottom: 1320, Right: 514
A face is located at pixel location Top: 1231, Left: 822, Bottom: 1320, Right: 912
A face is located at pixel location Top: 1513, Left: 748, Bottom: 1621, Right: 856
A face is located at pixel location Top: 1430, Left: 1369, Bottom: 1537, Right: 1477
A face is located at pixel location Top: 1382, Left: 1763, Bottom: 1489, Rig

...


1049: ../images/49376652_1595877350127066_r.jpg


1050: ../images/49379282_1594881600169860_r.jpg

A face is located at pixel location Top: 0, Left: 247, Bottom: 376, Right: 632

1051: ../images/49380804_159388395281869_r.jpg


1052: ../images/49382152_1593889619200288_r.jpg


1053: ../images/49399560_1594034690384192_r.jpg

A face is located at pixel location Top: 366, Left: 1005, Bottom: 552, Right: 1191
A face is located at pixel location Top: 192, Left: 365, Bottom: 415, Right: 588
A face is located at pixel location Top: 304, Left: 77, Bottom: 490, Right: 262
A face is located at pixel location Top: 167, Left: 637, Bottom: 390, Right: 860
A face is located at pixel location Top: 632, Left: 838, Bottom: 786, Right: 992

1054: ../images/49418720_1594039312772647_r.jpg

A face is located at pixel location Top: 68, Left: 439, Bottom: 291, Right: 662
A face is located at pixel location Top: 222, Left: 221, Bottom: 407, Right: 407

1055: ../images/49427288_1594061089582363_r.jpg


1056


1093: ../images/49668766_1594872363213355_r.jpg

A face is located at pixel location Top: 134, Left: 98, Bottom: 455, Right: 419

1094: ../images/49676134_1594868553476952_r.jpg


1095: ../images/49687830_1595662217621129_r.jpg

A face is located at pixel location Top: 204, Left: 0, Bottom: 590, Right: 375

1096: ../images/49694458_1595048182610089_r.jpg


1097: ../images/49697382_1598535213426985_r.jpg


1098: ../images/49723674_1594998863962386_r.jpg

A face is located at pixel location Top: 30, Left: 476, Bottom: 184, Right: 631
A face is located at pixel location Top: 168, Left: 282, Bottom: 297, Right: 411

1099: ../images/49729860_159501913224858_r.jpg

A face is located at pixel location Top: 320, Left: 349, Bottom: 587, Right: 617
A face is located at pixel location Top: 409, Left: 111, Bottom: 676, Right: 379
A face is located at pixel location Top: 41, Left: 439, Bottom: 84, Right: 482
A face is located at pixel location Top: 18, Left: 505, Bottom: 54, Right: 541

1100: ../i

A face is located at pixel location Top: 876, Left: 1481, Bottom: 951, Right: 1556
A face is located at pixel location Top: 1131, Left: 1499, Bottom: 1221, Right: 1589
A face is located at pixel location Top: 1035, Left: 2217, Bottom: 1143, Right: 2325
A face is located at pixel location Top: 1022, Left: 1380, Bottom: 1111, Right: 1469
A face is located at pixel location Top: 488, Left: 2022, Bottom: 550, Right: 2084
A face is located at pixel location Top: 964, Left: 2432, Bottom: 1071, Right: 2540
A face is located at pixel location Top: 320, Left: 1722, Bottom: 395, Right: 1796
A face is located at pixel location Top: 312, Left: 1498, Bottom: 387, Right: 1572
A face is located at pixel location Top: 1111, Left: 1748, Bottom: 1201, Right: 1837
A face is located at pixel location Top: 478, Left: 1821, Bottom: 553, Right: 1896
A face is located at pixel location Top: 312, Left: 2394, Bottom: 387, Right: 2468
A face is located at pixel location Top: 486, Left: 1597, Bottom: 561, Right: 

A face is located at pixel location Top: 167, Left: 439, Bottom: 390, Right: 662

1204: ../images/50356198_1597179325698899_r.jpg


1205: ../images/50358630_1597187416659500_r.jpg


1206: ../images/50373412_1597232875497118_r.jpg


1207: ../images/50374160_1597239362400891_r.jpg

A face is located at pixel location Top: 119, Left: 247, Bottom: 504, Right: 632

1208: ../images/50378906_1597331428924861_r.jpg

A face is located at pixel location Top: 502, Left: 428, Bottom: 1168, Right: 1093

1209: ../images/50380564_1597828160413305_r.jpg


1210: ../images/50381574_1597353373879162_r.jpg

A face is located at pixel location Top: 439, Left: 438, Bottom: 706, Right: 706

1211: ../images/50421614_1597373424974882_r.jpg

A face is located at pixel location Top: 155, Left: 229, Bottom: 229, Right: 303
A face is located at pixel location Top: 204, Left: 486, Bottom: 279, Right: 560
A face is located at pixel location Top: 319, Left: 163, Bottom: 426, Right: 270
A face is located at pixel loca

A face is located at pixel location Top: 80, Left: 330, Bottom: 187, Right: 438

1262: ../images/50981960_1599292030890869_r.jpg


1263: ../images/51006242_1599508157578441_r.jpg

A face is located at pixel location Top: 80, Left: 114, Bottom: 142, Right: 176

1264: ../images/51070260_1599611615237944_r.jpg


1265: ../images/_1570777223151741_r.jpg

A face is located at pixel location Top: 196, Left: 185, Bottom: 285, Right: 275
A face is located at pixel location Top: 196, Left: 536, Bottom: 271, Right: 610
A face is located at pixel location Top: 126, Left: 374, Bottom: 216, Right: 464

1266: ../images/_1573259462445773_r.jpg


1267: ../images/_157367379359475_r.jpg

A face is located at pixel location Top: 336, Left: 474, Bottom: 398, Right: 536
A face is located at pixel location Top: 56, Left: 79, Bottom: 163, Right: 187

1268: ../images/_1575572376364999_r.jpg

A face is located at pixel location Top: 997, Left: 3424, Bottom: 2147, Right: 4575

1269: ../images/_1576179891712278_r

In [79]:
df.tail()

,File,Count,Location,Emotions
1298,../images/_1585675653429775_r.jpg,3,"[(1301, 2340, 1568, 2073), (1427, 1118, 1889, ...","[Fear, Fear, Surprise]"
1299,../images/_1585827935361429_r.jpg,0,[],NaN
1300,../images/_1586570361983422_r.jpg,1,"[(297, 296, 426, 167)]",[Happy]
1301,../images/_1586808008203382_r.jpg,2,"[(631, 664, 674, 621), (626, 613, 662, 577)]","[Happy, Happy]"
1302,../images/_1586905091542756_r.jpg,0,[],NaN


In [83]:
isnotnan(df.Emotions)

NameError: name 'isnotnan' is not defined